# UAV stacks for Seward Pen. sites

In [2]:
import numpy as np
import os
import glob
import math

import rioxarray

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import Normalize

import sys
sys.path.insert(0, '/home/pmontesa/code/pygeotools')
from pygeotools.lib import filtlib, iolib, malib, warplib
sys.path.insert(0, '/home/pmontesa/code/imview')
from imview.lib import pltlib
#import scipy.ndimage

In [3]:
import rasterio
from rasterio.plot import show_hist, show
from rasterio.windows import Window
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

In [4]:
def rescale_pct_clip(array, pct=[1,80]):
    '''Re-scales data values of an array from 0-1 with percentiles'''
    array_min, array_max = np.nanpercentile(array,pct[0]), np.nanpercentile(array,pct[1])
    clip = (array - array_min) / (array_max - array_min)
    clip[clip>1]=1
    clip[clip<0]=0
    return clip

def rescale_multiband_for_plot(fn, rescaled_multiband_fn, bandlist = [4,3,2], pct=[5,95], nodata=-9999.0):
    
    # add a reduced res: https://gis.stackexchange.com/questions/434441/specifying-target-resolution-when-resampling-with-rasterio
    
    with rasterio.open(fn, "r+") as src1:
        
        src1.nodata = nodata
        arr_list = []
        for band in bandlist:
            arr = src1.read(band)
            arr_list.append(arr)
            
        with rasterio.open(rescaled_multiband_fn, 'w+',
                driver='GTiff',
                dtype= rasterio.float32,
                count=3,
                crs = src1.crs,
                width=src1.width,
                height=src1.height,
                transform=src1.transform,
                nodata=src1.nodata

            ) as dst:

            for i, band in enumerate(bandlist): 
                V = rescale_pct_clip(src1.read(band), pct=pct)
                dst.write(V,i+1)
                
    return rescaled_multiband_fn

### UAV extents for which you want IFSAR

In [6]:
EXPEDITION_ID = 'sewardpen2024'
OUTDIR = f'/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/{EXPEDITION_ID}'

In [15]:
%%time
RGB_DIR = f'{OUTDIR}/RGB_COG'
DSM_DIR = f'{OUTDIR}/DSM_COG'
DTM_DIR = f'{OUTDIR}/DTM_COG'
print(RGB_DIR)
fn_uavrgb_list = glob.glob(f'{RGB_DIR}/*group1.tif')
fn_uavdsm_list = glob.glob(f'{DSM_DIR}/*.tif')
fn_uavdtm_list = glob.glob(f'{DTM_DIR}/*.tif')

/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG
CPU times: user 2.41 ms, sys: 641 µs, total: 3.05 ms
Wall time: 2.49 ms


In [18]:
sorted(fn_uavdsm_list)

['/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/council_01_20240812_100m_dsm.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/council_02_20240812_100m_dsm.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/kougarok_01_20240811_100m_dsm.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/kougarok_02_20240811_100m_dsm.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/kougarok_03_20240811_100m_dsm.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/kougarok_04_20240813_100m_dsm.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/kougarok_05_20240814_100m_agl_dsm.tif',
 '/explore/

In [19]:
sorted(fn_uavrgb_list)

['/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/council_01_20240812_100m_transparent_mosaic_group1.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/council_02_20240812_100m_transparent_mosaic_group1.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/kougarok_01_20240811_100m_transparent_mosaic_group1.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/kougarok_02_20240811_100m_transparent_mosaic_group1.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/kougarok_03_20240811_100m_transparent_mosaic_group1.tif',
 '/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/kougarok_04_20240813_100m_transparent_mosaic_group1.tif',
 '/explore/nobackup/peop

In [10]:
fn_uavrescaled_list = [rescale_multiband_for_plot(fn, fn.replace('.tif', '_rescaled_multiband_temp.tif'), bandlist = [3,2,1], pct=[5,95], nodata=-9999.0) for fn in fn_uav_list] 

In [20]:
fn_ifsar_list = glob.glob('/explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_d*m_20221222.tif')

In [21]:
uav_ds_lists = []
for i,uavrgb_fn in enumerate(sorted(fn_uavrgb_list)):
    print(uavrgb_fn)
    uavdsm_fn = sorted(fn_uavdsm_list)[i]
    uavdtm_fn = sorted(fn_uavdtm_list)[i]
    ds_list = warplib.memwarp_multi_fn([uavrgb_fn, uavdsm_fn, uavdtm_fn] + fn_ifsar_list, extent=uavrgb_fn, res=0.5, t_srs=uavrgb_fn, r='cubic')
    uav_ds_lists.append(ds_list)

/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/council_01_20240812_100m_transparent_mosaic_group1.tif

Warping all inputs to the following:
Resolution: 0.5
Extent: [538570.8319, 7174772.051420001, 538966.4745, 7175284.893580001]
Projection: '+proj=utm +zone=3 +datum=WGS84 +units=m +no_defs'
Resampling alg: cubic

1 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/council_01_20240812_100m_transparent_mosaic_group1.tif
nl: 1026 ns: 791 res: 0.500
0

Warning 1: council_01_20240812_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes
Warning 1: council_01_20240812_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes
Warning 1: council_01_20240812_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes


...10...20...30...40...50...60...70...80...90...2 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/council_01_20240812_100m_dsm.tif
nl: 1026 ns: 791 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...3 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DTM_COG/council_01_20240812_100m_dtm.tif
nl: 1026 ns: 791 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...4 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dtm_20221222.tif
nl: 1026 ns: 791 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...5 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dsm_20221222.tif
nl: 1026 ns: 791 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90.../explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/council_02_20240812_100m_transparent_mosaic_group

Warning 1: council_02_20240812_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes
Warning 1: council_02_20240812_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes
Warning 1: council_02_20240812_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes


100 - done.
0...10...20...30...40...50...60...70...80...90...2 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/council_02_20240812_100m_dsm.tif
nl: 1008 ns: 472 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...3 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DTM_COG/council_02_20240812_100m_dtm.tif
nl: 1008 ns: 472 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...4 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dtm_20221222.tif
nl: 1008 ns: 472 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...5 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dsm_20221222.tif
nl: 1008 ns: 472 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90.../explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/kougarok_01_20240811_100m_transparen

...

..10...20...30...40...50...60...70...80...90...2 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/kougarok_05_20240814_100m_agl_dsm.tif
nl: 675 ns: 495 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...3 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DTM_COG/kougarok_05_20240814_100m_agl_dtm.tif
nl: 675 ns: 495 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...4 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dtm_20221222.tif
nl: 675 ns: 495 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...5 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dsm_20221222.tif
nl: 675 ns: 495 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90.../explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/kougarok_06_20240814_100m_transparent_mosaic

Warning 1: kougarok_06_20240814_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes
Warning 1: kougarok_06_20240814_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes
Warning 1: kougarok_06_20240814_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes


100 - done.
0...10...20...30...40...50...60...70...80...90...2 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/kougarok_06_20240814_100m_dsm.tif
nl: 917 ns: 263 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...3 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DTM_COG/kougarok_06_20240814_100m_dtm.tif
nl: 917 ns: 263 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...4 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dtm_20221222.tif
nl: 917 ns: 263 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...5 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dsm_20221222.tif
nl: 917 ns: 263 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90.../explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/RGB_COG/teller_01_20240810_100m_transparent_mo

Warning 1: teller_01_20240810_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes
Warning 1: teller_01_20240810_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes
Warning 1: teller_01_20240810_100m_transparent_mosaic_group1.tif: This file used to have optimizations in its layout, but those have been, at least partly, invalidated by later changes


100 - done.
0...10...20...30...40...50...60...70...80...90...2 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DSM_COG/teller_01_20240810_100m_dsm.tif
nl: 503 ns: 378 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...3 of 5: /explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/sewardpen2024/DTM_COG/teller_01_20240810_100m_dtm.tif
nl: 503 ns: 378 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...4 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dtm_20221222.tif
nl: 503 ns: 378 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...5 of 5: /explore/nobackup/projects/dem/AK_IFSAR/alaska_ifsar_dsm_20221222.tif
nl: 503 ns: 378 res: 0.500
100 - done.
0...10...20...30...40...50...60...70...80...90...

In [1]:
from matplotlib.colors import LinearSegmentedColormap
forest_ht_cmap = LinearSegmentedColormap.from_list('forest_ht', ['#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850','#005a32'], 12)

In [ ]:
def hillshade(array,azimuth,angle_altitude):
    #https://github.com/rveciana/introduccion-python-geoespacial/blob/master/hillshade.py
    azimuth = 360.0 - azimuth 

    x, y = np.gradient(array)
    slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    azimuthrad = azimuth*np.pi/180.
    altituderad = angle_altitude*np.pi/180.

    shaded = np.sin(altituderad)*np.sin(slope) + np.cos(altituderad)*np.cos(slope)*np.cos((azimuthrad - np.pi/2.) - aspect)

    return 255*(shaded + 1)/2
def do_scalebar(ax):
    return ax.add_artist(ScaleBar(
        dx=1,
        units="m",
        dimension="si-length",
         length_fraction=0.15,
        scale_formatter=lambda value, unit: f' {value} m ',
        location='lower left'
        ))
def do_colorbar(ax, fig, image_hidden, label, SIZE='1.5%' ):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size=SIZE, pad=0.15) #pad=-0.1)
    cb = fig.colorbar(image_hidden, cax=cax, orientation='vertical', extend='max')
    cb.set_label(label)

In [ ]:
# Determine the grid size
cols = len(uav_ds_lists[0:3])  # Number of columns in the grid
rows = 2

# Create a grid of subplots
fig, axes = plt.subplots(rows, cols, figsize=(15, 3.5 * rows))
axes = axes.flatten()  # Flatten in case we have a 2D array of axes

for i, ds_list in enumerate(uav_ds_lists[0:3]):
    ax = axes[i]
    if i <= cols:
        #map rgb
        #uav_rgb_ds = ds_list[0]
        
        #m_sr = ax.imshow(iolib.ds_getma(uav_rgb_ds))
        src_ds = rasterio.open(fn_uavrescaled_list[i])
        show(src_ds.read(), ax=ax, title=os.path.basename(fn_ifsar_list[i]) )
    else:
        ifsar_dtm_ma = iolib.ds_getma(ds_list[1])
        ifsar_dsm_ma = iolib.ds_getma(ds_list[2])
        ifsar_chm_ma = ifsar_dsm_ma - ifsar_dtm_ma
        hs_ma = hillshade(ifsar_dsm_ma,315,1)
        m_chm = ax.imshow(ifsar_chm_ma, cmap=forest_ht_cmap, clim=(0,5))
        m_chm = ax.imshow(hs_ma, cmap='gray', clim=malib.calcperc(hs_ma), alpha=0.5)
        do_colorbar(ax, fig, m_chm, 'Vegetation height [m]', SIZE='5%' )

plt.tight_layout()  